# Aethr - Active Learning Fine-Tune (Google Colab Web + Google Drive)

This notebook reads feedback packages and fine-tunes from existing best weights.

In [ ]:
from pathlib import Path
import os
from google.colab import drive

drive.mount('/content/drive')

PROJECT_ROOT = Path('/content/aethr')
REPO_URL = 'https://github.com/VishwaJaya01/aethr.git'

if not PROJECT_ROOT.exists():
    !git clone {REPO_URL} {PROJECT_ROOT}

os.chdir(PROJECT_ROOT)
!git pull
!pip install -q -r requirements.txt
!nvidia-smi

print('PROJECT_ROOT:', PROJECT_ROOT)

In [ ]:
from pathlib import Path
import json
import pandas as pd

DRIVE_WORKDIR = Path('/content/drive/MyDrive/aethr')
DRIVE_FEEDBACK_DIR = DRIVE_WORKDIR / 'data' / 'feedback'
DRIVE_FEEDBACK_DIR.mkdir(parents=True, exist_ok=True)

json_files = sorted(DRIVE_FEEDBACK_DIR.glob('*.json'))
print('Feedback JSON files:', len(json_files))

records = []
for json_file in json_files:
    payload = json.loads(json_file.read_text(encoding='utf-8'))
    records.append(
        {
            'json_file': json_file.name,
            'source_image': payload.get('source_image', ''),
            'saved_image': payload.get('saved_image', ''),
            'uncertain_count': payload.get('uncertain_count', 0),
            'note': payload.get('note', ''),
        }
    )

feedback_df = pd.DataFrame(records)
display(feedback_df.head(20))
print('Feedback records:', len(feedback_df))

manifest_path = DRIVE_FEEDBACK_DIR / 'feedback_manifest.csv'
feedback_df.to_csv(manifest_path, index=False)
print('Saved manifest:', manifest_path)

## Re-annotation Requirement

Before fine-tuning, re-annotate selected feedback images and export a YOLO dataset.

Expected Drive path in this notebook:

```text
/content/drive/MyDrive/aethr/data/raw/solar-panel-defects-feedback/data.yaml
```

In [ ]:
from pathlib import Path
from ultralytics import YOLO

DRIVE_WORKDIR = Path('/content/drive/MyDrive/aethr')
BASE_MODEL = DRIVE_WORKDIR / 'models' / 'best.pt'
FEEDBACK_DATA_YAML = (
    DRIVE_WORKDIR / 'data' / 'raw' / 'solar-panel-defects-feedback' / 'data.yaml'
)
RUNS_DIR = DRIVE_WORKDIR / 'runs'
RUNS_DIR.mkdir(parents=True, exist_ok=True)

if not BASE_MODEL.exists():
    raise FileNotFoundError(f'Base model missing: {BASE_MODEL}')
if not FEEDBACK_DATA_YAML.exists():
    raise FileNotFoundError(f'Feedback data.yaml missing: {FEEDBACK_DATA_YAML}')

model = YOLO(str(BASE_MODEL))
results = model.train(
    data=str(FEEDBACK_DATA_YAML),
    epochs=20,
    imgsz=640,
    batch=16,
    patience=10,
    project=str(RUNS_DIR),
    name='aethr_feedback_finetune',
    device=0,
)

print('Fine-tuning complete. Save dir:', results.save_dir)

In [ ]:
from pathlib import Path
import shutil

save_dir = Path(results.save_dir)
best_ft = save_dir / 'weights' / 'best.pt'
if not best_ft.exists():
    raise FileNotFoundError(f'Fine-tuned best.pt missing: {best_ft}')

DRIVE_MODELS_DIR = Path('/content/drive/MyDrive/aethr/models')
DRIVE_MODELS_DIR.mkdir(parents=True, exist_ok=True)

best_feedback_path = DRIVE_MODELS_DIR / 'best_feedback.pt'
shutil.copy2(best_ft, best_feedback_path)
print('Saved fine-tuned model to:', best_feedback_path)

# Set to True if you want feedback model to replace production best.pt.
REPLACE_PRODUCTION_BEST = False
if REPLACE_PRODUCTION_BEST:
    shutil.copy2(best_ft, DRIVE_MODELS_DIR / 'best.pt')
    print('Replaced production best.pt with feedback fine-tuned model.')
else:
    print('Production best.pt unchanged.')